In [127]:
# some important imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import os

In [70]:
with open("dataset.txt", "r", encoding='utf-8') as file:
    text = file.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

In [71]:
def tokenize(string):
    return [chars.index(c) for c in string]
def decode(l):
    return "".join([chars[i] for i in l])

data = torch.tensor(tokenize(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [72]:
train_data = data[:int(0.9*len(data))]
val_data = data[int(0.9*len(data)):]

In [73]:
def get_batch(batch_size, seq_len, split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - seq_len, (batch_size,))
    x = torch.stack([data[i : i + seq_len] for i in ix])
    y = torch.stack([data[i + 1 : i + seq_len + 1] for i in ix])
    return x, y

In [74]:
vocab_size = len(chars)
batch_size = 64
seq_len = 400
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n_embed = 512
n_heads = 8
n_layers = 6
dropout = 0.2
max_steps = 10000
learning_rate = 6e-5
eval_iters = 200
eval_interval = 1000
folder_path = 'weights'

In [100]:
class MultiHeadAttention(nn.Module):
    """Multiple Heads of Self Attention in parallel"""

    def __init__(self, n_embed, seq_len, n_heads, dropout=0.1):
        super().__init__()
        self.n_heads = n_heads
        self.head_size = n_embed // n_heads
        self.scale = self.head_size ** -0.5

        self.key = nn.Linear(n_embed, n_embed, bias=False)
        self.query = nn.Linear(n_embed, n_embed, bias=False)
        self.value = nn.Linear(n_embed, n_embed, bias=False)
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

        self.register_buffer('tril', torch.tril(torch.ones(seq_len, seq_len)))

    def forward(self, x):
        B, T, C = x.shape

        # Linear projections
        k = self.key(x).view(B, T, self.n_heads, self.head_size).transpose(1, 2)
        q = self.query(x).view(B, T, self.n_heads, self.head_size).transpose(1, 2)
        v = self.value(x).view(B, T, self.n_heads, self.head_size).transpose(1, 2)

        # Scaled dot-product attention
        affinities = (q @ k.transpose(-2, -1)) * self.scale
        affinities = affinities.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        affinities = F.softmax(affinities, dim=-1)
        affinities = self.dropout(affinities)

        out = affinities @ v
        out = out.transpose(1, 2).contiguous().view(B, T, C)

        # Final linear projection
        out = self.proj(out)
        return self.dropout(out)

class FFN(nn.Module):
    def __init__(self, n_embed, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class DecoderBlock(nn.Module):
    def __init__(self, n_embed, n_heads, seq_len, dropout):
        super().__init__()
        self.sa = MultiHeadAttention(n_embed, seq_len, n_heads)
        self.ffn = FFN(n_embed, dropout)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = self.ln1(x + self.sa(x))
        x = self.ln2(x + self.ffn(x))
        return x

class GPT(nn.Module):
    def __init__(self, vocab_size, seq_len, n_embed, n_heads, n_layers, dropout):
        super().__init__()
        self.seq_len = seq_len
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(seq_len, n_embed)
        self.blocks = nn.Sequential(*[DecoderBlock(n_embed, n_heads, seq_len, dropout) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)
    
    def forward(self, x, targets=None):
        B, T = x.shape
        tok_emb = self.token_embedding_table(x)
        pos_emb = self.position_embedding_table(torch.arange(T, device=x.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            logits = logits.view(B * T, -1)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_len):
        for _ in range(max_len):
            idx_cond = idx[:, -self.seq_len:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)

            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [101]:
model = GPT(vocab_size, seq_len, n_embed, n_heads, n_layers, dropout).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
def save_checkpoint(model, optimizer, step, folder='weights', filename='checkpoint_step_{step}.pth'):
    os.makedirs(folder, exist_ok=True)
    checkpoint = {
        'step': step,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(checkpoint, os.path.join(folder, filename.format(step=step)))

def load_checkpoint(model, optimizer, folder='weights'):
    if not os.path.exists(folder):
        print("No checkpoints found.")
        return 0  # No checkpoint to load

    checkpoints = [f for f in os.listdir(folder) if f.startswith('checkpoint_step_') and f.endswith('.pth')]
    if not checkpoints:
        print("No checkpoints found.")
        return 0  # No checkpoint to load

    latest_checkpoint = max(checkpoints, key=lambda f: int(f.split('_')[2].split('.')[0]))
    checkpoint_path = os.path.join(folder, latest_checkpoint)
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    print(f"Loaded checkpoint from {checkpoint_path} at step {checkpoint['step']}")
    return checkpoint['step']

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for i in range(eval_iters):
            X, Y = get_batch(batch_size, seq_len, split)
            logits, loss = model(X, Y)
            losses[i] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

start_step = load_checkpoint(model, optimizer, folder=folder_path)
for step in range(start_step, max_steps):
    
    if (step + 1) % eval_interval == 0:
        save_checkpoint(model, optimizer, step + 1, folder_path)
        losses = estimate_loss(model)
        print(f"step {step + 1} ,, train loss: {losses['train']:.4f} ,, val loss: {losses['val']:.4f}")
    
    xb, yb = get_batch(batch_size, seq_len, 'train')
    xb, yb = xb.to(device), yb.to(device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long).to(device)
print(decode(model.generate(context, 500)[0].tolist()))